In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import pandas as pd
import re
from IPython.display import display, HTML
import spacy
#import en_core_web_lg
from spacy.lang.en.stop_words import STOP_WORDS
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer


##for clustering
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk import word_tokenize

##To download packages directly from jupyter notebook
import sys ##execute the pip installs if you are running this for the first time
# !{sys.executable} -m pip install gensim
# !{sys.executable} -m pip install pyLDAvis

# display(HTML("<style>div.output_scroll { width: 80em; }</style>"))

%matplotlib inline  
import matplotlib.pyplot as plt

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Franck\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Franck\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
os.getcwd()

'F:\\Dropbox\\Travaux_JAOTOMBO\\These_Sante_Publique\\Mimic3\\Codes'

In [4]:
DATA_PATH = os.path.join(os.getcwd(), "..","Readmission","Data","icu_only")

'F:\\Dropbox\\Travaux_JAOTOMBO\\These_Sante_Publique\\Mimic3\\Codes\\..\\Readmission\\Data\\icu_only'

In [6]:
raw_df = pd.read_feather(os.path.join(DATA_PATH,"df_cleaned"))

In [7]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33316 entries, 0 to 33315
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   index            33316 non-null  int64         
 1   subject_id       33316 non-null  int64         
 2   hadm_id          33316 non-null  int64         
 3   admittime        33316 non-null  datetime64[ns]
 4   dischtime        33316 non-null  datetime64[ns]
 5   first_careunit   33316 non-null  object        
 6   last_careunit    33316 non-null  object        
 7   age              33316 non-null  float64       
 8   gender           33316 non-null  object        
 9   marital_status   31754 non-null  object        
 10  insurance        33316 non-null  object        
 11  diagnosis        33315 non-null  object        
 12  admission_type   33316 non-null  object        
 13  text             33316 non-null  object        
 14  next_readmit_dt  33316 non-null  float

In [8]:
df = raw_df.copy()

In [ ]:
#df["text"][0]

In [9]:
def clean_text(x):
    x = " ".join(x.split())
    x= " ".join((" ".join(x.split("[**"))).split("**]"))
    x = re.sub(r"\([^()]*\)", "", x)
    key_value_strip =(x.split(":"))
    ##remove all sub strings which have a length lesser than 50 characters
    string = " ".join([sub_unit for sub_unit in key_value_strip if len(sub_unit)>50])
    x = re.sub(r"(\d+)+(\.|\))", "", string)## remove all serialization eg 1. 1)
    x = re.sub(r"(\*|\?|=)+", "", x) ##removing all *, ? and =
    x = re.sub(r"\b(\w+)( \1\b)+", r"\1", x) ## removing consecutive duplicate words
    x = x.replace("FOLLOW UP", "FOLLOWUP")
    x = x.replace("FOLLOW-UP", "FOLLOWUP")
    x = re.sub(r"(\b)(f|F)(irst)(\b)?[\d\-\d]*(\s)*(\b)?(n|N)(ame)[\d\-\d]*(\s)*[\d\-\d]*(\b)","",x)##remove firstname
    x = re.sub(r"(\b)(l|L)(ast)(\b)?[\d\-\d]*(\s)*(\b)?(n|N)(ame)[\d\-\d]*(\s)*[\d\-\d]*(\b)", "", x)
    x = re.sub(r"(\b)(d|D)\.?(r|R)\.?(\b)", "", x) #remove Dr abreviation
    x = re.sub(r"(\b)(m|M)\.?(d|D)\.?(\b)", "", x) #remove M.D. abreviation
    x = re.sub(r"([^A-Za-z0-9\s](\s)){2,}", "", x)##remove consecutive punctuations

    return(x.replace("  ", " "))

In [10]:
%time df["text"] = df["text"].apply(lambda x: clean_text(x))

Wall time: 2min 8s


In [11]:
display(HTML(df.head(2).to_html()))

In [ ]:
spacy.prefer_gpu
nlp = spacy.load("en_core_sci_sm")

def clean_text(text):
    sentence = []
    for w in text:
    # if it's not a punctuation mark nor number, add it to our article!
        if w.text != '\n' and not w.is_punct and not w.like_num and not w.is_digit and not w.is_stop :
        # we add the lematized version of the word but do not remove the stopwords (and not w.is_stop)
            sentence.append(w.lemma_)
    strings = ' '.join(sentence)
    return strings   

In [12]:
#spacy.prefer_gpu
nlp = spacy.load("en_core_sci_sm")

def clean_more_text(text):
    sentence = []
    for w in text:
    # if it's not a punctuation mark nor number, add it to our article!
        if w.text != '\n' and not w.is_punct and not w.like_num and not w.is_digit and not w.is_stop :
        # we add the lematized version of the word but do not remove the stopwords (and not w.is_stop)
            sentence.append(w.lemma_)
    strings = ' '.join(sentence)
    return strings   

In [ ]:
# text = df["text"][0]
# text

In [ ]:
# doc = nlp(text)
# doc

In [ ]:
# clean = clean_text(doc)
# clean

In [13]:
df['doc'] = df["text"].apply(nlp)
df['cleaned'] = df['doc'].apply(clean_more_text)

In [ ]:
display(HTML(df.head(2).to_html()))

In [ ]:
topics = df["text"].apply(lambda text:list(filter(None,[x.strip() for x in re.findall(r"\b[A-Z\s]+\b",text)])))
topics = topics.tolist() 
#combine all items from topics and find unique elements 
topics = sorted(set([sub_topic.strip() for topic in topics for sub_topic in topic]))
topics

In [ ]:
topics = df["TEXT"].apply(lambda text:list(filter(None,[x.strip() for x in re.findall(r"\b[A-Z\s]+\b",text)])))
topics = topics.tolist() 
#combine all items from topics and find unique elements 
topics = sorted(set([sub_topic.strip() for topic in topics for sub_topic in topic]))
topics

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True)
from unidecode import unidecode

In [ ]:
# Find out the source !!!

med_stopwords = {'street',
'address',
'required',
'powder',
'developed',
'intermittently',
'weekly',
'later',
'echo',
'commands',
'comfort',
'back',
'ultimately',
'complete',
'daughter',
'nutrition',
'range',
'knee',
'subsequently',
'summary',
'upon',
'service',
'completed',
'consistent',
'pattern',
'woman',
'address',
'inhaled',
'times',
'count',
'number',
'underwent',
'post',
'oral',
'general',
'recommend',
'goal',
'remains',
'need',
'report',
'solution',
'female',
'exploratory',
'level',
'poor',
'aggressive',
'support',
'determined',
'hand',
'instructions',
'follow',
'rehabilitation',
'large',
'decreased',
'stay',
'four',
'hours',
'intake',
'name',
'patient',
'access',
'good',
'comfortable',
'type',
'initials',
'external',
'percent',
'descending',
'output',
'collection',
'stop',
'presented',
'unit',
'name',
'positive',
'number',
'dictated',
'line',
'plus',
'date',
'active',
'done',
'records',
'state',
'month',
'notable',
'requiring',
'factor',
'current',
'male',
'history',
'number',
'completed',
'tenderness',
'ward',
'name',
'office',
'port',
'impression',
'trace',
'improvement',
'group',
'scan',
'given',
'patient',
'laboratory',
'right',
'upper',
'however',
'patient',
'volume',
'limited',
'suggestive',
'presents',
'year',
'also',
"mg",
"ml",
"mm",
"unchanged",
"normal",
"admissions",
"social"
}

In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^a-z\s]')
STOPWORDS = set(stopwords.words('english'))

In [ ]:
include_stopwords = set(topics)| med_stopwords
STOPWORDS |= include_stopwords

In [ ]:
def text_prepare(text) :
    """
        text: a string        
        return: modified initial string
    """
        
    text = text.lower() # lowercase text
    text = unidecode((text))
    text = REPLACE_BY_SPACE_RE.sub(" ", text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub(" ",text) # delete symbols which are in BAD_SYMBOLS_RE from text
    #text =  " ".join([stemmer.stem(x) for x in text.split()])
    #text =  " ".join([x for x in text.split()])
    text =  " ".join([x for x in text.split()if x not in STOPWORDS]) # delete stopwords from text
   
    return text

In [ ]:
cleaned = text_prepare(clean)
cleaned

In [ ]:
df["clean"] = df["cleaned"].apply(lambda x: text_prepare(x))

In [ ]:
display(HTML(df.head(2).to_html()))

In [ ]:
# df.to_feather(os.path.join(DATA_PATH,"30_READM_CLEAN_DISCHARGE_ICU"))